## Data Wrangling
- V1 on 9/22/2025 8am ET
- V2 on 10/4/2025 11am PT
- V3 on 10/5/2025 12pm PT
- V4 on 10/7/2025 8am PT

In [ ]:
!pip install --quiet gdown pandas
!pip install boto3

In [ ]:
import boto3
from google.colab import userdata
import io
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.utils import resample

#### Random Seed and Sample Size

In [ ]:
RANDOM_STATE = 5678
SAMPLE_SIZE = 100  # Size per age bucket

Download from S3 for CV 23.0

In [ ]:
# Establish session with credentials
bucket = "asrelder-data"
session = boto3.Session(
    aws_access_key_id=userdata.get("AWS_ACCESS"),
    aws_secret_access_key=userdata.get("AWS_SECRET"),
    region_name="us-east-1"
)
s3 = session.client("s3")

In [ ]:
# S3 base path
prefix = "common_voice/23/cv-corpus-23.0-2025-09-05/en/"
filename = "train.tsv"
obj = s3.get_object(Bucket=bucket, Key=prefix+filename)
file_bytes = io.BytesIO(obj["Body"].read())
print(f"Loaded {filename} into memory...")

In [ ]:
train_df = pd.read_csv(file_bytes, sep="\t")
print(f"train_df.shape: {train_df.shape}")
train_df.head(2)

In [ ]:
train_df.to_csv("common_voices_23_full.csv")

In [ ]:
# S3 base path
prefix = "common_voice/23/cv-corpus-23.0-2025-09-05/en/"
filename = "clip_durations.tsv"
obj = s3.get_object(Bucket=bucket, Key=prefix+filename)
file_bytes = io.BytesIO(obj["Body"].read())
print(f"Loaded {filename} into memory...")

In [ ]:
clip_duration_df = pd.read_csv(file_bytes, sep="\t")
print(f"clip_duration_df.shape: {clip_duration_df.shape}")
clip_duration_df.head(2)

In [ ]:
clip_duration_df.to_csv("common_voices_23_clip_durations.csv")

#### Filter to only rows having age

In [ ]:
train_with_non_null_age = train_df[train_df["age"].notna()]
print(f"Removing nulls reduces to {train_with_non_null_age.shape[0]}/{train_df.shape[0]} rows")
# Removing nulls reduces to 814443/1142930 rows

#### Filter to only rows having 1+ upvote and 0 downvotes

In [ ]:
train_with_validated_votes = train_with_non_null_age[
    (train_with_non_null_age["up_votes"] >= 1) &
    (train_with_non_null_age["down_votes"] == 0)
]
print(f"Removing nulls reduces to {train_with_validated_votes.shape[0]}/{train_with_non_null_age.shape[0]} rows")
# Removing nulls reduces to 699129/814443 rows
train_with_validated_votes.head(2)

In [ ]:
train_with_validated_votes.to_csv("common_voices_23_train_with_validated_votes.csv")

#### Separate out dfs for each age bucket
- Exclude teens age bucket for COPPA considerations

In [ ]:
# Get dataframe per age bucket of equal number of rows
age_order = ["twenties", "thirties", "fourties", "fifties", "sixties", "seventies", "eighties", "nineties"]
dfs_list = []
for age_bucket in age_order:
    temp_df = train_with_validated_votes[train_with_validated_votes["age"] == age_bucket]
    print(f"{age_bucket} filtered rows: {temp_df.shape[0]}")
    sample_df = temp_df.sample(n=SAMPLE_SIZE, random_state=RANDOM_STATE)
    print(f"{age_bucket} sample size rows: {sample_df.shape[0]}")
    dfs_list.append(sample_df)

In [ ]:
# Consolidate to single dataframe
sample_df = pd.concat(dfs_list, ignore_index=True)
print(f"Combined sample df has shape: {sample_df.shape}")
sample_df.head(2)

In [ ]:
sample_df.to_csv("common_voices_23_sample1.csv")

#### Big 60/40 Set of Below 60/60 and Above
- Exclude teens age bucket for COPPA considerations

In [ ]:
# Create new column
younger = ["twenties", "thirties", "fourties", "fifties"]
older = ["sixties", "seventies", "eighties", "nineties"]
train_with_validated_votes["age_group"] = np.select(
    [
        train_with_validated_votes["age"].isin(older),
        train_with_validated_votes["age"].isin(younger)
    ],
    ["Older", "Younger"],
    default=None
)

# Balanced 50/50 between Younger/Older -- undersample (downsample the larger group)
train_with_certain_ages = train_with_validated_votes[train_with_validated_votes["age_group"].notna()]
print(f"Filtering to two age groups reduces to {train_with_certain_ages.shape[0]}/{train_with_validated_votes.shape[0]} rows")

older_df = train_with_certain_ages[train_with_certain_ages["age_group"] == "Older"]
younger_df = train_with_certain_ages[train_with_certain_ages["age_group"] == "Younger"]
print(f"Older: {older_df.shape[0]}, Younger: {younger_df.shape[0]}")
younger_sample = younger_df.sample(n=len(older_df), random_state=RANDOM_STATE)
print(f"Older: {older_df.shape[0]}, Younger (sampled): {younger_sample.shape[0]}")
train_with_balanced_ages = pd.concat(
    [older_df, younger_sample]
).sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)
print(f"Final balanced dataset: {train_with_balanced_ages.shape[0]} rows")
train_with_balanced_ages.head(2)

In [ ]:
train_with_balanced_ages.to_csv("common_voices_23_balanced_on_60.csv")

#### Download silences.csv from S3
- Was made on EC2 and copied to S3

In [ ]:
# S3 base path
prefix = "common_voice/23/"
filename = "silences.csv"
obj = s3.get_object(Bucket=bucket, Key=prefix+filename)
file_bytes = io.BytesIO(obj["Body"].read())
print(f"Loaded {filename} into memory...")

In [ ]:
silences_df = pd.read_csv(file_bytes, sep=",")
silences_df["path"] = silences_df["file"].str.split("/").str[-1]
print(f"silences_df.shape: {silences_df.shape}")
silences_df.head(2)

In [ ]:
# Check for overlap between silences.file and train.path
merged_df = silences_df.merge(train_df, how="left", on="path", indicator=True)
print(merged_df["_merge"].value_counts())

In [ ]:
silences_df[
    ["path", "file", "start_sec", "duration_sec"]
].to_csv("common_voices_23_silences.csv")